# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import psycopg2
from sql_queries import airport_insert, demographic_insert, immigration_insert, temperature_insert

### Step 1: Scope the Project and Gather Data

#### Scope 
This projects aims to enrich the US I94 immigration data with further data such as demographics and temperature data to have a wider basis for analysis on the immigration data.

#### 1.1. I94 Immigration Data

This data comes from the US National Tourism and Trade Office. A data dictionary is included in the workspace. This is where the data comes from. There's a sample file so you can take a look at the data in csv format before reading it all in. You do not have to use the entire dataset, just use what you need to accomplish the goal you set at the beginning of the project.

In [2]:
# Read in the data here
i94_path = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df_i94 = pd.read_sas(i94_path, 'sas7bdat', encoding="ISO-8859-1")
df_i94.head(5)

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


#### 1.2. World Temperature Data

This dataset came from Kaggle.

In [3]:
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
df_temp = pd.read_csv(fname)
df_temp.head(5)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [153]:
# find all unique country codes in temperature data to find used name for United States 
#set(df_temp["Country"].values)

In [4]:
# Get the data for United States

df_temp_us = df_temp[df_temp["Country"] == "United States"]
df_temp_us.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
47555,1820-01-01,2.101,3.217,Abilene,United States,32.95N,100.53W
47556,1820-02-01,6.926,2.853,Abilene,United States,32.95N,100.53W
47557,1820-03-01,10.767,2.395,Abilene,United States,32.95N,100.53W
47558,1820-04-01,17.989,2.202,Abilene,United States,32.95N,100.53W
47559,1820-05-01,21.809,2.036,Abilene,United States,32.95N,100.53W


#### 1.3. U.S. City Demographic Data

This data comes from OpenSoft.

In [5]:
df_demographics = pd.read_csv("data/us-cities-demographics.csv", delimiter=";")
df_demographics.head(5)

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


#### 1.4. Airport Code Table

This is a simple table of airport codes and corresponding cities.

In [6]:
df_airport_codes = pd.read_csv("data/airport-codes_csv.csv")
df_airport_codes.head(5)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

In [7]:
# Describe i94 data
#df_i94.describe()

In [8]:
# Describe USA Temperature data
#df_temp_us.describe()

In [9]:
# Describe Airport Codes data
#df_airport_codes.describe()

In [11]:
# Describe Demographics data
#df_demographics.describe()

### Step 2.1: Clean the data
Document steps necessary to clean the data
- 2.1.1. I94 data
- 2.1.2. Temperature data
- 2.1.1. Airport data
- 2.1.1. Demographics data


### 2.1.1. Clean up i94 data

In [7]:
# Get port locations from SAS text file

with open("data/I94_SAS_Labels_Descriptions.SAS") as f:
    content = f.readlines()
content = [x.strip() for x in content]
ports = content[302:962]
splitted_ports = [port.split("=") for port in ports]

port_codes = [x[0].replace("'","").strip() for x in splitted_ports]
port_locations = [x[1].replace("'","").strip() for x in splitted_ports]
port_cities = [x.split(",")[0] for x in port_locations]
port_states = [x.split(",")[-1] for x in port_locations]

df_port_locations = pd.DataFrame({"port_code" : port_codes, "port_city": port_cities, "port_state": port_states})
df_port_locations.head(20)

,port_code,port_city,port_state
0,ALC,ALCAN,AK
1,ANC,ANCHORAGE,AK
2,BAR,BAKER AAF - BAKER ISLAND,AK
3,DAC,DALTONS CACHE,AK
4,PIZ,DEW STATION PT LAY DEW,AK
5,DTH,DUTCH HARBOR,AK
6,EGL,EAGLE,AK
7,FRB,FAIRBANKS,AK
8,HOM,HOMER,AK
9,HYD,HYDER,AK


In [8]:
# print first and last element in dict to check if all lines in file are covered

print(f"First port in SAS file: {df_port_locations['port_city'].values[0]}, last port {df_port_locations['port_city'].values[-1]}")

irregular_ports_df = df_port_locations[df_port_locations["port_city"] == df_port_locations["port_state"]]
irregular_ports = list(set(irregular_ports_df["port_code"].values))

print(irregular_ports)

First port in SAS file: ALCAN, last port No PORT Code (OSN)
['74S', 'JBQ', 'JMZ', 'FSC', 'FTB', 'WAS', 'ZZZ', 'PHF', 'WA5', 'CPX', 'BUS', 'X44', 'AUH', 'SCH', 'GPI', 'NK', 'HRL', 'Y62', 'BCM', 'YGF', 'CHN', 'OLM', 'STN', 'IAG', 'FRG', 'MTH', 'OAI', 'A2A', 'AMT', 'PHN', 'RYY', 'JIG', 'AKT', 'NC8', 'CXO', 'X96', 'JSJ', '888', 'OTS', 'DEC', 'XXX', 'XNA', 'UNK', 'SP0', 'GMT', '.GA', 'SUS', 'ISP', 'PLB', '5T6', 'DRV', '060', 'DAY', 'AG', 'W55', 'NGL', 'JFA', 'MAP', 'NYL', 'WTR', 'ATW', 'BKF', 'PFN', 'OGS', 'EGE', 'VMB', 'CP', 'CLX', 'LIT', 'ASI', 'MAA', 'PCW', 'T01', 'TIW', 'GAC', 'OSN', 'ADU']


In [9]:
# drop all irregular ports from i94 data

print(f"i94 data contains {len(df_i94)} rows before cleaning.")
df_i94_filtered = df_i94[~df_i94["i94port"].isin(irregular_ports)]

print(f"i94 data contains {len(df_i94_filtered)} rows after removing irregular ports.")

df_i94_filtered.drop(columns=["insnum", "entdepu", "occup", "visapost"], inplace=True)
df_i94_filtered.dropna(inplace=True)

print(f"i94 data contains {len(df_i94_filtered)} rows after removing NaN values.")

i94 data contains 3096313 rows before cleaning.
i94 data contains 2995590 rows after removing irregular ports.


/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


i94 data contains 2306754 rows after removing NaN values.


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [12]:
#df_i94_filtered['i94addr'] = df_i94_filtered['i94addr'].str.replace('""',)

In [11]:

# Save the usa data into a csv, which will be used to load the immigration table
#df_i94_filtered.to_csv('data/i94_filtered.csv', sep=';', index = False ,float_format='%11.2f' ,encoding='utf-8') 
#df = pd.read_csv(filename, sep=',').replace('"','', regex=True)

df_i94_filtered.to_csv('data/i94_filtered1.csv', sep=';', index = False ,float_format='%11.2f')


### 2.1.2. Clean up the Temperature data

In [13]:
# Check for the list of columns having null data
chk_nan = df_temp_us.isnull().sum()
chk_nan

dt                                   0
AverageTemperature               25765
AverageTemperatureUncertainty    25765
City                                 0
Country                              0
Latitude                             0
Longitude                            0
dtype: int64

In [14]:
# clear missing temperature values
df_temp_us.dropna(inplace=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [15]:
# Save the usa data into a csv, which will be used to load the temperature table
df_temp_us.to_csv('data/usa_temperature.csv',sep=';', index=False,float_format='%11.2f')

### 2.1.3. Clean up Airport data

In [16]:
# Check the count before clean up
df_airport_codes.count()

ident           55075
type            55075
name            55075
elevation_ft    48069
continent       27356
iso_country     54828
iso_region      55075
municipality    49399
gps_code        41030
iata_code        9189
local_code      28686
coordinates     55075
dtype: int64

In [17]:
# Drop the NaN records for iata_code

df_airport_codes.dropna(subset=['iata_code'], inplace=True)
df_airport_codes.head(5)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
223,03N,small_airport,Utirik Airport,4.0,OC,MH,MH-UTI,Utirik Island,K03N,UTK,03N,"169.852005, 11.222"
440,07FA,small_airport,Ocean Reef Club Airport,8.0,NaN,US,US-FL,Key Largo,07FA,OCA,07FA,"-80.274803161621, 25.325399398804"
594,0AK,small_airport,Pilot Station Airport,305.0,NaN,US,US-AK,Pilot Station,NaN,PQS,0AK,"-162.899994, 61.934601"
673,0CO2,small_airport,Crested Butte Airpark,8980.0,NaN,US,US-CO,Crested Butte,0CO2,CSE,0CO2,"-106.928341, 38.851918"
1088,0TE7,small_airport,LBJ Ranch Airport,1515.0,NaN,US,US-TX,Johnson City,0TE7,JCY,0TE7,"-98.62249755859999, 30.251800537100003"


In [18]:
# Delete deuplicate data from airport codes
df_airport_codes = df_airport_codes.drop_duplicates(subset='iata_code', keep='last')

In [19]:
# Check the count after clean up
df_airport_codes.count()

ident           9042
type            9042
name            9042
elevation_ft    8693
continent       6085
iso_country     9011
iso_region      9042
municipality    8282
gps_code        8424
iata_code       9042
local_code      2979
coordinates     9042
dtype: int64

In [21]:
#df_airport_codes.loc[df_airport_codes['iata_code'] == 'BCK']

In [27]:
#df_airport_codes

#### 2.1.4. Clean up Demographics data

In [20]:
# Check for the list of columns having null data

chk_nan = df_demographics.isnull().sum()
chk_nan

City                       0
State                      0
Median Age                 0
Male Population            3
Female Population          3
Total Population           0
Number of Veterans        13
Foreign-born              13
Average Household Size    16
State Code                 0
Race                       0
Count                      0
dtype: int64

In [21]:
# Drop NaN records from the data frame
df_demographics.dropna(inplace=True)

In [22]:
# Re-check the values after the modification

chk_nan = df_demographics.isnull().sum()
chk_nan

City                      0
State                     0
Median Age                0
Male Population           0
Female Population         0
Total Population          0
Number of Veterans        0
Foreign-born              0
Average Household Size    0
State Code                0
Race                      0
Count                     0
dtype: int64

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

##### Tables:
| table name | columns | description | type |
| ------- | ---------- | ----------- | ---- |
| airports | iata_code - name - type - local_code - coordinates - city | stores information related to airports | dimension table |
| demographics | city - state - media_age - male_population - female_population - total_population - num_veterans - foreign_born - average_household_size - state_code - race - count | stores demographics data for cities | dimension table |
| immigrations | cicid - year - month - cit - res - iata - arrdate - mode - addr - depdate - bir - visa - coun- dtadfil - visapost - occup - entdepa - entdepd - entdepu - matflag - biryear - dtaddto - gender - insnum - airline - admnum - fltno - visatype | stores all i94 immigrations data | fact table |
| temperature | timestamp - average_temperature - average_temperatur_uncertainty - city - country - latitude - longitude | stores temperature information | dimension table |



#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

1. Create tables by executing `create_tables.py`.
2. Join city to airports data.
3. Insert data.

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model


- 4.1.1. Insert data into Airports table
- 4.1.2. Insert data into Demographics table
- 4.1.3. Insert data into Temperature table
- 4.1.4. Insert data into Immigration table

### After running create_tables.py, insert the data into the database

In [1]:
#import psycopg2
#from sql_queries import airport_insert, demographic_insert, immigration_insert, temperature_insert

In [23]:
# Create connection to the "sparkifydb" database
def connect_db():
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    return cur, conn


In [37]:
cur, conn = connect_db()

### 4.1.1. Insert data into Airport table

In [24]:
df_airport_codes = df_airport_codes.merge(df_port_locations, left_on="iata_code", right_on="port_code")

df_airport_codes.drop(columns=["port_code"], inplace=True)
df_airport_codes = df_airport_codes[["iata_code", "name", "type", "local_code", "coordinates", "port_city", "elevation_ft", "continent", "iso_country", "iso_region", "municipality", "gps_code"]]

In [25]:
df_airport_codes.head()

,iata_code,name,type,local_code,coordinates,port_city,elevation_ft,continent,iso_country,iso_region,municipality,gps_code
0,TKI,Tokeen Seaplane Base,seaplane_base,57A,"-133.32699585, 55.9370994568",TOKEEN,NaN,NaN,US,US-AK,Tokeen,57A
1,BKF,Lake Brooks Seaplane Base,seaplane_base,5Z9,"-155.77699279785, 58.554798126221",No PORT Code (BKF),36.0,NaN,US,US-AK,Katmai National Park,5Z9
2,AXB,Maxson Airfield,small_airport,89NY,"-75.90034, 44.312002",ALEXANDRIA BAY,340.0,NaN,US,US-NY,Alexandria Bay,89NY
3,FRE,Fera/Maringe Airport,small_airport,NaN,"159.576996, -8.1075",FRESNO,NaN,OC,SB,SB-IS,Fera Island,AGGF
4,ANZ,Angus Downs Airport,small_airport,NaN,"132.2748, -25.0325",ANZALDUAS,1724.0,OC,AU,AU-NT,Angus Downs Station,NaN


In [282]:
#df_airport_codes.loc[df_airport_codes['iata_code'] == 'TKI']

#### 4.1.1.1. Fix the Non-ASCII data issue in the Airport code

* Airport table contains Non-ASCII values in the "Name" and "Municipality" column
* This data needs to be cleaned
* Replace Nan values with 'NA'
* Convert Non-ASCII values to ASCII values before loading the data into the immigrations table to avoid encoding error

In [26]:
# Function to check for non-ascii data
# look out for Name and Municipality columns

def is_not_ascii(string):
    return string is not None and any([ord(s) >= 128 for s in string])

df_airport_codes[df_airport_codes['name'].apply(is_not_ascii)]

,iata_code,name,type,local_code,coordinates,port_city,elevation_ft,continent,iso_country,iso_region,municipality,gps_code
16,NOR,NorÃ°fjÃ¶rÃ°ur Airport,small_airport,NaN,"-13.746399879455566, 65.13189697265625",NORFOLK,13.0,EU,IS,IS-7,NorÃ°fjÃ¶rÃ°ur,BINF
53,BOO,BodÃ¸ Airport,large_airport,NaN,"14.365300178527832, 67.26920318603516",BOOTHBAY HARBOR,42.0,EU,NO,NO-18,BodÃ¸,ENBO
63,WPB,Port BergÃ© Airport,small_airport,NaN,"47.623587, -15.584286",WEST PALM BEACH,213.0,AF,MG,MG-M,Port BergÃ©,FMNG
65,FRK,FrÃ©gate Island Airport,small_airport,NaN,"55.950001, -4.583",FREDERIKSTED,610.0,AF,SC,SC-15,FrÃ©gate Island,FSSF
332,MAD,Adolfo SuÃ¡rez MadridâBarajas Airport,large_airport,NaN,"-3.56264, 40.471926",MADAWASKA,1998.0,EU,ES,ES-M,Madrid,LEMD
334,CHR,"ChÃ¢teauroux-DÃ©ols ""Marcel Dassault"" Airport",medium_airport,NaN,"1.721111, 46.860278",CHRISTIANSTED,529.0,EU,FR,FR-F,ChÃ¢teauroux/DÃ©ols,LFLX
338,PEV,PÃ©cs-PogÃ¡ny Airport,medium_airport,NaN,"18.240996, 45.990898",PORT EVERGLADES,1000.0,EU,HU,HU-BA,PÃ©cs-PogÃ¡ny,LHPP
344,BGC,BraganÃ§a Airport,medium_airport,NaN,"-6.70713, 41.8578",BRIDGEPORT,2241.0,EU,PT,PT-04,BraganÃ§a,LPBG
345,BCM,BacÄu Airport,medium_airport,NaN,"26.91029930114746, 46.52190017700195",No PORT Code (BCM),607.0,EU,RO,RO-BC,BacÄu,LRBC
347,CND,Mihail KogÄlniceanu International Airport,medium_airport,NaN,"28.488300323486328, 44.36220169067383",KOGALNICEANU,353.0,EU,RO,RO-CT,ConstanÅ£a,LRCK


In [27]:
# Fill in the NA column of municipality with a string

df_airport_codes["municipality"].fillna("NA", inplace = True)

In [285]:
# checking for non-ascii data

'''def is_not_ascii(string):
    return string is not None and any([ord(s) >= 128 for s in string])

df_airport_codes[df_airport_codes['municipality'].apply(is_not_ascii)]
'''

"def is_not_ascii(string):\n    return string is not None and any([ord(s) >= 128 for s in string])\n\ndf_airport_codes[df_airport_codes['municipality'].apply(is_not_ascii)]\n"

In [28]:
# Function to convert non-ASCII to ASCII characters
def convertToAscii(string):
    return string.encode("ascii","ignore").decode()
    
    '''string_nonASCII = string
    string_encode = string_nonASCII.encode("ascii", "ignore")
    #print(string_encode)
    string_decode = string_encode.decode()
    return string_decode'''
    

In [241]:
#def remove_non_ascii(s):
#    return "".join(c for c in s if ord(c)<128)
#df_airport_codes['name'] = df_airport_codes['name'].apply(remove_non_ascii)

In [29]:
# Apply the function on name and municipality columns of the data frame

df_airport_codes['name'] = df_airport_codes['name'].apply(convertToAscii)

In [30]:
df_airport_codes['municipality'] = df_airport_codes['municipality'].apply(convertToAscii)

In [31]:
# Check the data after modification
df_airport_codes.head(20)

,iata_code,name,type,local_code,coordinates,port_city,elevation_ft,continent,iso_country,iso_region,municipality,gps_code
0,TKI,Tokeen Seaplane Base,seaplane_base,57A,"-133.32699585, 55.9370994568",TOKEEN,NaN,NaN,US,US-AK,Tokeen,57A
1,BKF,Lake Brooks Seaplane Base,seaplane_base,5Z9,"-155.77699279785, 58.554798126221",No PORT Code (BKF),36.0,NaN,US,US-AK,Katmai National Park,5Z9
2,AXB,Maxson Airfield,small_airport,89NY,"-75.90034, 44.312002",ALEXANDRIA BAY,340.0,NaN,US,US-NY,Alexandria Bay,89NY
3,FRE,Fera/Maringe Airport,small_airport,NaN,"159.576996, -8.1075",FRESNO,NaN,OC,SB,SB-IS,Fera Island,AGGF
4,ANZ,Angus Downs Airport,small_airport,NaN,"132.2748, -25.0325",ANZALDUAS,1724.0,OC,AU,AU-NT,Angus Downs Station,NaN
5,COB,Coolibah Airport,small_airport,NaN,"130.96200561523438, -15.548299789428711",COBURN GORE,NaN,OC,AU,AU-NT,Coolibah,NaN
6,CRY,Carlton Hill Airport,small_airport,NaN,"128.53399658203125, -15.501899719238281",CARBURY,NaN,OC,AU,AU-WA,Carlton Hill,NaN
7,SWB,Shaw River Airport,small_airport,NaN,"119.36199951171875, -21.510299682617188",SWANTON,NaN,OC,AU,AU-WA,Shaw River,NaN
8,EPT,Eliptamin Airport,small_airport,ELP,"141.6779, -5.0412",EASTPORT MUNICIPAL,4825.0,OC,PG,PG-SAN,Eliptamin,AYEL
9,HNN,Honinabi Airport,small_airport,HBI,"142.1771, -16.2457",HANNAH,452.0,OC,PG,PG-WPD,Honinabi,AYHH


In [32]:
# Re-check the values before modification

chk_nan = df_airport_codes.isnull().sum()
chk_nan

iata_code         0
name              0
type              0
local_code      239
coordinates       0
port_city         0
elevation_ft     17
continent       307
iso_country       1
iso_region        0
municipality      0
gps_code         27
dtype: int64

In [33]:
# Fill elevation_ft column with 0 for NaN values
df_airport_codes['elevation_ft'] = df_airport_codes['elevation_ft'].fillna(0)


In [34]:
chk_nan = df_airport_codes.isnull().sum()
chk_nan

iata_code         0
name              0
type              0
local_code      239
coordinates       0
port_city         0
elevation_ft      0
continent       307
iso_country       1
iso_region        0
municipality      0
gps_code         27
dtype: int64

In [35]:
# Save the usa data into a csv, which will be used to load the immigration table
df_airport_codes.to_csv('data/df_airport_codes.csv',sep=';', index=False, encoding = "utf-8")

In [296]:
# Load the Airport table with the csv file, as this is faster compared to loading from data frame

'''with open('data/df_airport_codes.csv', 'r') as f:
    next(f) # Skip the header row.
    cur.copy_from(f, 'airports', sep=';')

conn.commit()'''


"with open('data/df_airport_codes.csv', 'r') as f:\n    next(f) # Skip the header row.\n    cur.copy_from(f, 'airports', sep=';')\n\nconn.commit()"

In [38]:
# Load the Airport table

for index, row in df_airport_codes.iterrows():
    cur.execute(airport_insert, list(row.values))
    conn.commit()

### 4.1.2. Insert data into Demographics table

In [39]:
# Load the Demographics table

for index, row in df_demographics.iterrows():
    cur.execute(demographic_insert, list(row.values))
    conn.commit()

### 4.1.3. Insert data into Temperature table

In [40]:
# Load the Temperature table with the csv file, as this is faster compared to loading from data frame

with open('data/usa_temperature.csv', 'r') as f:
    next(f) # Skip the header row.
    cur.copy_from(f, 'temperature', sep=';')

conn.commit()

In [ ]:
'''for index, row in df_temp_us.iterrows():
    #if index == 100:
        #print(f"index: {index}")
    cur.execute(temperature_insert, list(row.values))
    conn.commit()
'''    

### 4.1.4. Insert data into Immigration table

In [41]:
# Load the Immigrations table with the csv file, as this is faster compared to loading from data frame

with open('data/i94_filtered.csv', 'r') as f:
    next(f) # Skip the header row.
    cur.copy_from(f, 'immigrations', sep=';')

conn.commit()

In [24]:
'''for index, row in df_i94_filtered.iterrows():
    if index == 10000:
        print(row.values)
    cur.execute(immigration_insert, list(row.values))
    conn.commit()'''

'for index, row in df_i94_filtered.iterrows():\n    if index == 10000:\n        print(row.values)\n    cur.execute(immigration_insert, list(row.values))\n    conn.commit()'

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [42]:
# Perform quality checks here

cur.execute("SELECT COUNT(*) FROM airports")
airport_count = cur.fetchone()
print(f"airport_count: {airport_count}")
conn.commit()

if cur.rowcount < 1:
    print("No data found in table airports")
    
cur.execute("SELECT COUNT(*) FROM demographics")
demographics_count = cur.fetchone()
print(f"demographics_count: {demographics_count}")


conn.commit()
if cur.rowcount < 1:
    print("No data found in table demographics")
    
cur.execute("SELECT COUNT(*) FROM immigrations")
immigrations_count = cur.fetchone()
print(f"immigrations_count: {immigrations_count}")

conn.commit()
if cur.rowcount < 1:
    print("No data found in table immigrations")
    
cur.execute("SELECT COUNT(*) FROM temperature")
temperature_count = cur.fetchone()
print(f"temperature_count: {temperature_count}")
conn.commit()
if cur.rowcount < 1:
    print("No data found in table temperature")

airport_count: (550,)
demographics_count: (2875,)
immigrations_count: (2306754,)
temperature_count: (661524,)


In [43]:
conn.close()

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
   * Use Spark to process the data efficiently in a distributed way e.g. with EMR.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
   * Use Airflow and create a DAG that performs the logic of the described pipeline.
 * The database needed to be accessed by 100+ people.
   * Use RedShift to have the data stored in a way that it can efficiently be accessed by many people.